## Solving the set of partitioning problem

### Select the desire instance and build the hamiltonian

In the folder "instance_benchmark" you`ll find all the instances used in the paper, for this test example you can choose whatever you in the next cell.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Sampler
import warnings
warnings.filterwarnings('ignore')


from _utility import *

# select the instance
w, c, A = read_instance('instances_benchmark/14.1.txt')

# biulding the hamiltonian
parameters = (w, c, A)
H_op = build_hamiltonian(parameters)

# solving using gurobi
sol = sp_gurobi(w, A)
ref_value = sp_objective(w, sol)


print('Reference value:', ref_value)
print('Reference solution:', sol) 

Reference value: 215.0
Reference solution: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0]


### QCE-ACF Experiment

In [2]:
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_aer.noise import NoiseModel, depolarizing_error


estimator = AerSimulator()

## for a noise model test use this one
noise_model = NoiseModel()


# setting error to the gate 'cx'
error = depolarizing_error(0.1, 2)
error_meas = depolarizing_error(0.1, 1)

noise_model.add_all_qubit_quantum_error(error, ['cx'])
noise_model.add_all_qubit_quantum_error(error_meas, "measure")
basis_gates = noise_model.basis_gates

noisy_estimator = AerSimulator( 
                    noise_model = noise_model,
                    basis_gates = basis_gates
                    )

In [3]:
import qce_acf

res = qce_acf.minimize(estimator=estimator,hamiltonian=H_op, generations=10_000, population=4, ref_value=ref_value, tol= 1e-6, A= A)

print("Gurobi solution",sol)


Step: 11/10000 | Depth: 2 | Value: 215.0000000 | Key: 0000000000110000
Reached target tolerance at generation 11

Optimization completed in 1.33 seconds
Best value: 215.0000000
Best circuit depth: 2
Best Key: 00000000001100
Gurobi solution [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0]


### QAOA Experiment

In [4]:
# Import necessary libraries for QAOA implementation
from qiskit_algorithms.utils import algorithm_globals
from qiskit.primitives import Sampler
from qiskit_aer.primitives import Sampler as AerSampler
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA

# Lists to store optimization progress
values = []       # Stores energy values during optimization
eval_count = []   # Stores evaluation count/steps

# Callback function to track optimization progress
def save_results(step, parameters, value, std):
    eval_count.append(step)
    values.append(value)
    print(f"Run: Step: {step} Expected value: {value}", end="\r", flush=True)
    
# Initialize exact statevector simulator
sampler = Sampler()

sampler_noisy = AerSampler(backend_options={"noise_model": noise_model, "basis_gates": basis_gates},
                        run_options={"shots": 1042, "seed": 42})

# Set random seed for reproducibility
algorithm_globals.random_seed = 100

# Initialize COBYLA optimizer
optimizer = COBYLA()

# Create QAOA instance with 1 repetition and the callback function
qaoa = QAOA(sampler, optimizer, reps=1, callback=save_results)

# Run QAOA to find minimum eigenvalue of the Hamiltonian
result = qaoa.compute_minimum_eigenvalue(H_op)
print(result)


{   'aux_operators_evaluated': None,1.18411851284+0j)
    'best_measurement': {   'bitstring': '00000000001100',
                            'probability': np.float64(0.0005360917507226),
                            'state': 12,
                            'value': np.complex128(215+0j)},
    'cost_function_evals': 28,
    'eigenstate': {0: np.float64(6.0510317718e-05), 1: np.float64(4.5210946595e-05), 2: np.float64(1.5277854979e-05), 3: np.float64(1.8843718859e-05), 4: np.float64(1.3612510376e-05), 5: np.float64(2.2939955614e-05), 6: np.float64(1.1059328886e-05), 7: np.float64(8.7627295462e-05), 8: np.float64(8.8041056747e-05), 9: np.float64(0.00011832367133), 10: np.float64(5.701130227e-05), 11: np.float64(0.00010272605118), 12: np.float64(1.5623694501e-05), 13: np.float64(5.0520371496e-05), 14: np.float64(5.5147614912e-05), 15: np.float64(0.000192895651005), 16: np.float64(9.2803278748e-05), 17: np.float64(8.7805650969e-05), 18: np.float64(4.5845166332e-05), 19: np.float64(7.1473315